<a href="https://colab.research.google.com/github/shikhar00778/toxic-comment-classification/blob/master/Toxic_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/kaushaltrivedi/fast-bert.git

  Cloning https://github.com/kaushaltrivedi/fast-bert.git to /tmp/pip-req-build-c0hqv_gj
  Running command git clone -q https://github.com/kaushaltrivedi/fast-bert.git /tmp/pip-req-build-c0hqv_gj
     |████████████████████████████████| 133kB 9.3MB/s 
  Stored in directory: /tmp/pip-ephem-wheel-cache-555r3t6r/wheels/53/11/ef/67d507c8f4f05b843a6dd2c4f6360acdee4c79679c4ca36bc6
Successfully built fast-bert


In [6]:
%%writefile /usr/local/lib/python3.6/dist-packages/fast_bert/metrics.py
import numpy as np
from torch import Tensor
from sklearn.metrics import roc_curve, auc, hamming_loss, accuracy_score
import pdb

# def accuracy(out, labels):
#     outputs = np.argmax(out, axis=1)
#     return np.sum(outputs == labels)

def accuracy(y_pred:Tensor, y_true:Tensor):

    outputs = np.argmax(y_pred, axis=1)
    return np.mean(outputs.numpy() == y_true.detach().cpu().numpy())

def accuracy_multilabel(y_pred:Tensor, y_true:Tensor, sigmoid:bool=True):
    if sigmoid: y_pred = y_pred.sigmoid()
    outputs = np.argmax(y_pred, axis=1)
    real_vals = np.argmax(y_true, axis=1)
    return np.mean(outputs.numpy() == real_vals.numpy())

def accuracy_thresh(y_pred:Tensor, y_true:Tensor, thresh:float=0.5, sigmoid:bool=True):
    "Compute accuracy when `y_pred` and `y_true` are the same size."
    if sigmoid: y_pred = y_pred.sigmoid()
    return ((y_pred>thresh)==y_true.byte()).float().mean().item()
#     return np.mean(((y_pred>thresh)==y_true.byte()).float().cpu().numpy(), axis=1).sum()


def fbeta(y_pred:Tensor, y_true:Tensor, thresh:float=0.3, beta:float=2, eps:float=1e-9, sigmoid:bool=True):
    "Computes the f_beta between `preds` and `targets`"
    beta2 = beta ** 2
    if sigmoid: y_pred = y_pred.sigmoid()
    y_pred = (y_pred>thresh).float()
    y_true = y_true.float()
    TP = (y_pred*y_true).sum(dim=1)
    prec = TP/(y_pred.sum(dim=1)+eps)
    rec = TP/(y_true.sum(dim=1)+eps)
    res = (prec*rec)/(prec*beta2+rec+eps)*(1+beta2)
    return res.mean().item()

def roc_auc(y_pred: Tensor, y_true: Tensor):
    # ROC-AUC calcualation
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    
    y_true = y_true.detach().cpu().numpy()
    y_pred = y_pred.detach().cpu().numpy()
        
    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_true.ravel(), y_pred.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    
    return roc_auc["micro"]


Overwriting /usr/local/lib/python3.6/dist-packages/fast_bert/metrics.py


In [1]:
import torch
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing import text,sequence
from tqdm import tqdm
import torch
from torch import nn
from torch.utils import data
from torch.nn import functional as F
from fastai.train import Learner
from fastai.train import DataBunch
from fastai.callbacks import *
from fastai.metrics import *
import os
import random

import pytorch_pretrained_bert
from pytorch_pretrained_bert import BertModel
from pytorch_pretrained_bert.tokenization import BertTokenizer

from fast_bert.data import BertDataBunch
from fast_bert.learner import BertLearner
from fast_bert.metrics import accuracy , accuracy_multilabel

Using TensorFlow backend.


In [0]:

args = {
    "max_seq_length": 64,
    "do_lower_case": True,
    "train_batch_size": 32,
    "learning_rate": 6e-5,
    "num_train_epochs": 12.0,
    "warmup_proportion": 0.002,
    "local_rank": -1,
    "gradient_accumulation_steps": 1,
    "fp16": True,
    "loss_scale": 128
}

In [0]:
import logging
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

In [4]:
!ls

apex  fast-bert  sample_data


In [6]:
import logging
logging.basicConfig(level=logging.INFO)

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenized input
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = tokenizer.tokenize(text)

# Mask a token that we will try to predict back with `BertForMaskedLM`
masked_index = 8
tokenized_text[masked_index] = '[MASK]'
assert tokenized_text == ['[CLS]', 'who', 'was', 'jim', 'henson', '?', '[SEP]', 'jim', '[MASK]', 'was', 'a', 'puppet', '##eer', '[SEP]']

# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

INFO:pytorch_pretrained_bert.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt not found in cache, downloading to /tmp/tmpkdwxi_p4
100%|██████████| 231508/231508 [00:00<00:00, 1187024.99B/s]
INFO:pytorch_pretrained_bert.file_utils:copying /tmp/tmpkdwxi_p4 to cache at /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:pytorch_pretrained_bert.file_utils:creating metadata file for /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:pytorch_pretrained_bert.file_utils:removing temp file /tmp/tmpkdwxi_p4
INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f0006

In [7]:
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

# If you have a GPU, put everything on cuda
tokens_tensor = tokens_tensor.to('cuda')
segments_tensors = segments_tensors.to('cuda')
model.to('cuda')

# Predict hidden states features for each layer
with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors)
# We have a hidden states for each of the 12 layers in model bert-base-uncased
assert len(encoded_layers) == 12

INFO:pytorch_pretrained_bert.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz not found in cache, downloading to /tmp/tmp7htanrun
100%|██████████| 407873900/407873900 [00:11<00:00, 33997968.25B/s]
INFO:pytorch_pretrained_bert.file_utils:copying /tmp/tmp7htanrun to cache at /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:pytorch_pretrained_bert.file_utils:creating metadata file for /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:pytorch_pretrained_bert.file_utils:removing temp file /tmp/tmp7htanrun
INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb7

In [11]:
!git clone https://github.com/kaushaltrivedi/fast-bert.git

Cloning into 'fast-bert'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 95 (delta 33), reused 71 (delta 21), pack-reused 0
Unpacking objects: 100% (95/95), done.


In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

05/26/2019 16:46:02 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [6]:
!ls fast-bert/sample_data/multi_label_toxic_comments/label

labels.csv


In [0]:
device = torch.device('cuda')

# check if multiple GPUs are available
if torch.cuda.device_count() > 1:
    multi_gpu = True
else:
    multi_gpu = False

In [0]:
df = pd.read_csv('fast-bert/sample_data/multi_label_toxic_comments/data/train_sample.csv')

In [0]:
df_valid = pd.read_csv('fast-bert/sample_data/multi_label_toxic_comments/data/val_sample.csv')

In [17]:
df.head(1)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,c8779341e1728431,Japanese article / Merging two articles \nThe ...,0,0,0,0,0,0


In [0]:
df = df.rename(index=str, columns={"comment_text": "text"})

In [0]:
df_valid = df_valid.rename(index=str, columns={"comment_text": "text"})

In [0]:
!rm -r fast-bert/sample_data/multi_label_toxic_comments/data/train_sample.csv

In [0]:
!rm -r fast-bert/sample_data/multi_label_toxic_comments/data/val_sample.csv

In [0]:
df.to_csv('fast-bert/sample_data/multi_label_toxic_comments/data/train_sample.csv' , index = False)

In [0]:
df_valid.to_csv('fast-bert/sample_data/multi_label_toxic_comments/data/val_sample.csv' , index = False)

In [24]:
df.columns

Index(['id', 'text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [36]:
cd ..

/content


In [0]:
databunch = BertDataBunch('fast-bert/sample_data/multi_label_toxic_comments/data', 'fast-bert/sample_data/multi_label_toxic_comments/label', tokenizer, 
                          train_file='train_sample.csv', val_file='val_sample.csv',label_file='labels.csv',label_col=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult','identity_hate'],
                          bs=args['train_batch_size'], maxlen=args['max_seq_length'], 
                          multi_gpu=multi_gpu, multi_label=True)

In [9]:
logger

<Logger __main__ (INFO)>

In [28]:
cd apex

/content/apex


In [22]:
metrics = []
metrics.append({'name': 'accuracy_multilabel', 'function': accuracy_multilabel})

learner = BertLearner.from_pretrained_model(databunch, 'https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz', metrics, device, logger=logger, 
                                            finetuned_wgts_path=None, 
                                            is_fp16=args['fp16'], loss_scale=args['loss_scale'], 
                                            multi_gpu=multi_gpu,  multi_label=True)

05/26/2019 16:59:38 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
05/26/2019 16:59:38 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file /root/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmp7ojz1v7m
05/26/2019 16:59:42 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size

In [23]:
learner.fit(2, lr=args['learning_rate'], 
            schedule_type="warmup_cosine_hard_restarts")

05/26/2019 16:59:53 - INFO - __main__ -   Loss after epoch 0 - 0.24462223052978516
05/26/2019 16:59:53 - INFO - __main__ -   Running evaluation


05/26/2019 16:59:56 - INFO - __main__ -   Eval results:
05/26/2019 16:59:56 - INFO - __main__ -     eval_loss = 0.11819791793823242
05/26/2019 16:59:56 - INFO - __main__ -     metrics = {'accuracy_multilabel': 0.997}
05/26/2019 16:59:56 - INFO - __main__ -   --------------------------------------------------------------------------------
05/26/2019 17:00:01 - INFO - __main__ -   Loss after epoch 1 - 0.1369333267211914
05/26/2019 17:00:01 - INFO - __main__ -   Running evaluation


05/26/2019 17:00:04 - INFO - __main__ -   Eval results:
05/26/2019 17:00:04 - INFO - __main__ -     eval_loss = 0.08547687530517578
05/26/2019 17:00:04 - INFO - __main__ -     metrics = {'accuracy_multilabel': 0.997}
05/26/2019 17:00:04 - INFO - __main__ -   --------------------------------------------------------------------------------


In [0]:
texts = [
  "nothing",
  "Blah"
]

predictions = learner.predict_batch(texts)

In [34]:
predictions

[[('toxic', 0.03656005859375),
  ('insult', 0.01953125),
  ('obscene', 0.0182342529296875),
  ('severe_toxic', 0.005237579345703125),
  ('identity_hate', 0.003734588623046875),
  ('threat', 0.0036067962646484375)],
 [('toxic', 0.037109375),
  ('insult', 0.0199127197265625),
  ('obscene', 0.0184478759765625),
  ('severe_toxic', 0.005344390869140625),
  ('identity_hate', 0.003795623779296875),
  ('threat', 0.0036792755126953125)]]

In [31]:
!pip uninstall apex

Uninstalling apex-0.1:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/apex-0.1-py3.6-linux-x86_64.egg
Proceed (y/n)? y
  Successfully uninstalled apex-0.1


In [32]:
cd ..

/content


In [0]:
!rm -rf apex

In [34]:
!git clone https://github.com/NVIDIA/apex.git
%cd apex
!python setup.py install --cuda_ext --cpp_ext

Cloning into 'apex'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 4467 (delta 38), reused 40 (delta 20), pack-reused 4394
Receiving objects: 100% (4467/4467), 8.67 MiB | 10.63 MiB/s, done.
Resolving deltas: 100% (2866/2866), done.
/content/apex
torch.__version__  =  1.1.0

Compiling cuda extensions with
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130
from /usr/local/cuda/bin

running install
running bdist_egg
running egg_info
creating apex.egg-info
writing apex.egg-info/PKG-INFO
writing dependency_links to apex.egg-info/dependency_links.txt
writing top-level names to apex.egg-info/top_level.txt
writing manifest file 'apex.egg-info/SOURCES.txt'
writing manifest file 'apex.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_li

In [5]:
cd ..

/content


In [0]:
for step, batch in enumerate(databunch.train_dl):
  batch = tuple(t for t in batch)
  input_ids, input_mask, segment_ids, label_ids = batch
  break